In [1]:

import os
if os.path.split(os.getcwd())[-1]=='notebooks':
    os.chdir("../")

'Your base path is at: '+os.path.split(os.getcwd())[-1]

'Your base path is at: DS_COVID-19'

In [2]:

import subprocess
import os

import pandas as pd
import numpy as np

from datetime import datetime

import requests
import json 

def get_johns_hopkins():
    
    git_pull = subprocess.Popen( "/usr/bin/git pull" ,
                         cwd = os.path.dirname( 'data/raw/COVID-19/' ),
                         shell = True,
                         stdout = subprocess.PIPE,
                         stderr = subprocess.PIPE )
    (out, error) = git_pull.communicate()


    print("Error : " + str(error))
    print("out : " + str(out))


def get_current_data_germany():
    
    
    data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_Landkreisdaten/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

    json_object=json.loads(data.content)
    full_list=[]
    for pos,each_dict in enumerate (json_object['features'][:]):
        full_list.append(each_dict['attributes'])

    pd_full_list=pd.DataFrame(full_list)
    pd_full_list.to_csv('data/raw/NPGEO/GER_state_data.csv',sep=';')
    print(' Number of regions rows: '+str(pd_full_list.shape[0]))

if __name__ == '__main__':
    get_johns_hopkins()
    get_current_data_germany()

Error : b'From https://github.com/CSSEGISandData/COVID-19\n   f74f8d0cf5..dba1d42ad2  master     -> origin/master\n * [new branch]            automated-patch-for-delayed-data-for-Peru-Thailand-07-28-2022 -> origin/automated-patch-for-delayed-data-for-Peru-Thailand-07-28-2022\n   4e9c86e881..3023c6a417  web-data   -> origin/web-data\n'
out : b'Updating f74f8d0cf5..dba1d42ad2\nFast-forward\n README.md                                                             | 2 +-\n csse_covid_19_data/csse_covid_19_daily_reports/07-28-2022.csv         | 4 ++--\n .../time_series_covid19_confirmed_global.csv                          | 4 ++--\n .../csse_covid_19_time_series/time_series_covid19_deaths_global.csv   | 4 ++--\n 4 files changed, 7 insertions(+), 7 deletions(-)\n'
 Number of regions rows: 411


In [3]:

import pandas as pd
import numpy as np

from datetime import datetime


def store_relational_JH_data():
    
    data_path='data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    pd_raw=pd.read_csv(data_path)

    pd_data_base=pd_raw.rename(columns={'Country/Region':'country',
                      'Province/State':'state'})

    pd_data_base['state']=pd_data_base['state'].fillna('no')

    pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)


    pd_relational_model=pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )

    pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')

    pd_relational_model.to_csv('data/processed/COVID_relational_confirmed.csv',sep=';',index=False)
    print(' Number of rows stored: '+str(pd_relational_model.shape[0]))
    print(' Latest date is: '+str(max(pd_relational_model.date)))
if __name__ == '__main__':

    store_relational_JH_data()

 Number of rows stored: 261915
 Latest date is: 2022-07-28 00:00:00


In [4]:
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    
    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


def savgol_filter(df_input,column='confirmed',window=5):
    
    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0) 

    result=signal.savgol_filter(np.array(filter_in),
                           window, 
                           1)
    df_result[str(column+'_filtered')]=result
    return df_result

def rolling_reg(df_input,col='confirmed'):
    
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)



    return result




def calc_filtered_data(df_input,filter_on='confirmed'):
    

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() 

    pd_filtered_result=df_output[['state','country',filter_on]].groupby(['state','country']).apply(savgol_filter)

    
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
   
    return df_output.copy()





def calc_doubling_rate(df_input,filter_on='confirmed'):
    

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'


    pd_DR_result= df_input.groupby(['state','country']).apply(rolling_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

 
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output


if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

    pd_JH_data=pd.read_csv('data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
    pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()

 

    pd_result_larg=calc_filtered_data(pd_JH_data)
    pd_result_larg=calc_doubling_rate(pd_result_larg)
    pd_result_larg=calc_doubling_rate(pd_result_larg,'confirmed_filtered')


    mask=pd_result_larg['confirmed']>100
    pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)
    pd_result_larg.to_csv('data/processed/COVID_final_set.csv',sep=';',index=False)
    print(pd_result_larg[pd_result_larg['country']=='Mexico'].tail())

the test slope is: [2.]
             date state country  confirmed  confirmed_filtered  confirmed_DR  \
187471 2022-07-24    no  Mexico  6624310.0           6617527.8    372.866971   
187472 2022-07-25    no  Mexico  6628541.0           6637147.8   1188.269585   
187473 2022-07-26    no  Mexico  6628541.0           6651060.0   3132.654534   
187474 2022-07-27    no  Mexico  6686954.0           6669430.1    227.620975   
187475 2022-07-28    no  Mexico  6686954.0           6687800.2    228.287641   

        confirmed_filtered_DR  
187471             372.125524  
187472             389.456290  
187473             395.753646  
187474             412.148203  
187475             363.058998  


In [5]:
print(pd_result_larg[pd_result_larg['country']=='US'].tail())
print(pd_result_larg[pd_result_larg['country']=='United Kingdom'].tail())

             date state country   confirmed  confirmed_filtered  confirmed_DR  \
249044 2022-07-24    no      US  90410055.0          90502966.2   4981.487123   
249045 2022-07-25    no      US  90598955.0          90622784.8    902.088767   
249046 2022-07-26    no      US  90733680.0          90767152.8    559.789242   
249047 2022-07-27    no      US  90972856.0          90946575.5    485.521552   
249048 2022-07-28    no      US  91120218.0          91125998.2    470.547534   

        confirmed_filtered_DR  
249044             880.989447  
249045             830.613419  
249046             686.113285  
249047             560.725417  
249048             506.884444  
            date                        state         country  confirmed  \
1837  2022-07-28                     Anguilla  United Kingdom     3580.0   
15622 2022-07-28  Falkland Islands (Malvinas)  United Kingdom     1835.0   
26650 2022-07-28                     Guernsey  United Kingdom    29924.0   
21136 2022-07-28 

In [7]:
import pandas as pd
import numpy as np

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

import os
print(os.getcwd())
df_input_large=pd.read_csv('data/processed/COVID_final_set.csv',sep=';')


fig = go.Figure()

app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data

   The aim of this Dasboard is to use the skills provided in the lectures in order to present a dynamic graph. This graph shows how COVID-19 evolve through the time in different countries. Through this graph is possible to choose the countries of your preference and see how covid confirmed cases increases through the time. 

    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),


    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_input_large['country'].unique()],
        value=['Mexico', 'US','United Kingdom'], 
        multi=True
    ),

    dcc.Markdown('''
        ## Select Timeline of confirmed COVID-19 cases or the approximated doubling time
        '''),


    dcc.Dropdown(
    id='doubling_time',
    options=[
        {'label': 'Timeline Confirmed ', 'value': 'confirmed'},
        {'label': 'Timeline Confirmed Filtered', 'value': 'confirmed_filtered'},
        {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
        {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'},
    ],
    value='confirmed',
    multi=False
    ),

    dcc.Graph(figure=fig, id='main_window_slope')
])



@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('doubling_time', 'value')])
def update_figure(country_list,show_doubling):


    if 'doubling_rate' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days '
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed infected people (log-scale)'
              }


    traces = []
    for each in country_list:

        df_plot=df_input_large[df_input_large['country']==each]

        if show_doubling=='doubling_rate_filtered':
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.mean).reset_index()
        else:
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.sum).reset_index()
    


        traces.append(dict(x=df_plot.date,
                                y=df_plot[show_doubling],
                                mode='markers+lines',
                                opacity=0.7,
                                name=each
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#4d1313"),
                      },

                yaxis=my_yaxis
        )
    }

if __name__ == '__main__':

    app.run_server(debug=True, use_reloader=False)

/Users/isabelmartinez/DS_COVID-19
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


OSError: [Errno 48] Address already in use